In [ ]:
import os
from pathlib import Path
import subprocess
from colmap.scripts.python.read_write_model import read_images_binary, read_cameras_binary
from colmap.scripts.python.read_write_model import Image, Camera
import numpy as np
import json
import pickle

import matplotlib.pyplot as plt
import numpy as np
from scipy import stats


import logging
logging.getLogger().setLevel(logging.INFO)

In [ ]:
image_base_path = os.path.join("./reconstruct", "311/918", "colmap", "0")
image_bin_path = Path(os.path.join(image_base_path, "images.bin"))
all_bin_images = read_images_binary(image_bin_path)

In [ ]:
camera_bin_path = Path(os.path.join(image_base_path, "cameras.bin"))
all_bin_cameras = read_cameras_binary(camera_bin_path)

In [ ]:
n_points = []
for k, image in all_bin_images.items():
    n_points.append(len(image.point3D_ids))

n = np.array(n_points)
plt.hist(n, bins=int(np.sqrt(len(n))))

In [ ]:
image.qvec2rotmat()

In [ ]:
image = all_bin_images[972]
image.camera_id

In [ ]:
# https://github.com/colmap/colmap/blob/main/src/colmap/sensor/models.h
lengths = []
for idx, cam in all_bin_cameras.items():
    print(np.round(cam.params, 3))
    lengths.append(cam.params[0])
#all_bin_cameras[image.camera_id].model

In [ ]:
print(np.round(cam.params, 3))

In [ ]:
np.set_printoptions(suppress=True,precision=3)
print("Params are: f, cx, cz, k")
print(np.round(cam.params, 3))



# https://github.com/colmap/colmap/blob/main/src/colmap/sensor/models.h#L281
# what is k? https://github.com/colmap/colmap/issues/2628

In [ ]:

l = np.array(lengths)
plt.hist(l/100, bins=int(np.sqrt(len(l)*5)))
plt.xlim(0, 100)

print(np.median(l/100)) # theory by me: most smartphones actually use around the 20-25 mm focal length, 13-16 ultrawide and around 45-55 for zoom...
print(stats.mode(np.round(l/100, 0)))

In [ ]:
dir(all_bin_cameras[image.camera_id])

In [ ]:
print(image.qvec2rotmat())
R = image.qvec2rotmat()
# Rx(psi)
# Ry(theta)
# Rz(phi)

assert R[2,0] != 0, "the following method will not work now."

# R = Rz * Ry * Rx

In [ ]:
# y: Theta candidates because two solutions
theta1 = np.arcsin(R[2,0])
theta2 = np.pi - theta1
print(R[2,0])
print(theta1, theta2)

In [ ]:
# x: psi
psi = np.atan2(R[2,1], R[2,2])
print(psi)

#attention! if cos(theta) > 0 then positive , if cos(theta < 0) then negative

for t in [theta1, theta2]:
    assert not np.isclose(np.cos(t), 0), "must not be 0"

# for this case we compute two psi, except if cos(theta) == 0
psi1 = np.atan2(R[2,1]/np.cos(theta1), R[2,2]/np.cos(theta1))
psi2 = np.atan2(R[2,1]/np.cos(theta2), R[2,2]/np.cos(theta2))

print(psi1, psi2)

In [ ]:
# z: phi

phi = np.atan2(R[1,0], R[0,0])
print(phi)

for t in [theta1, theta2]:
    assert not np.isclose(np.cos(t), 0), "must not be 0"


phi1 = np.atan2(R[1,0]/np.cos(theta1), R[0,0]/np.cos(theta1))
phi2 = np.atan2(R[1,0]/np.cos(theta2), R[0,0]/np.cos(theta2))

print(phi1, phi2)

In [ ]:
# Two solutions in euler angles (radians?)

print(psi1, theta1, phi1)
print(psi2, theta2, phi2)

# Both solutions are valid

In [ ]:
if False and True:
    print("hi")

In [ ]:
# https://eecs.qmul.ac.uk/~gslabaugh/publications/euler.pdf
R = image.qvec2rotmat()
if np.isclose(R[2,1], 1) or np.isclose(R[2,1], -1):
    print("degenerate case of cos(theta) = 0")
    phi = 0 # can be anything...
    if np.islcose(R[2,0], -1):
        theta = np.pi/2
        psi = phi + np.atan2(R[0,1], R[0,2])
    else:
        theta = -np.pi/2
        psi = -phi + np.atan2(-R[0,1], -R[0,2])
else:
    print("non-degenerate case where cos(theta) != 0")
    theta1 = -np.asin(R[2,0])
    theta2 = np.pi - theta1

    psi1 = np.atan2(R[2,1]/np.cos(theta1), R[2,2]/np.cos(theta1))
    psi2 = np.atan2(R[2,1]/np.cos(theta2), R[2,2]/np.cos(theta2))

    phi1 = np.atan2(R[1,0]/np.cos(theta1), R[0,0]/np.cos(theta1))
    phi2 = np.atan2(R[1,0]/np.cos(theta2), R[0,0]/np.cos(theta2))

    psi = psi1
    theta = theta1
    phi = phi1


rot1 = np.array([psi1, theta1, phi1])
rot2 = np.array([psi2, theta2, phi2])
print(rot1)
print(rot2)

In [ ]:
np.degrees(rot1)

In [ ]:
def radians_from_rotation_matrix(rotation_matrix):
    # https://eecs.qmul.ac.uk/~gslabaugh/publications/euler.pdf
    R = rotation_matrix

    if np.isclose(R[2,1], 1) or np.isclose(R[2,1], -1):
        print("degenerate case of cos(theta) = 0")
        phi = 0 # can be anything...
        if np.islcose(R[2,0], -1):
            theta = np.pi/2
            psi = phi + np.atan2(R[0,1], R[0,2])
        else:
            theta = -np.pi/2
            psi = -phi + np.atan2(-R[0,1], -R[0,2])
    else:
        print("non-degenerate case where cos(theta) != 0")
        theta1 = -np.asin(R[2,0])
        theta2 = np.pi - theta1

        psi1 = np.atan2(R[2,1]/np.cos(theta1), R[2,2]/np.cos(theta1))
        psi2 = np.atan2(R[2,1]/np.cos(theta2), R[2,2]/np.cos(theta2))

        phi1 = np.atan2(R[1,0]/np.cos(theta1), R[0,0]/np.cos(theta1))
        phi2 = np.atan2(R[1,0]/np.cos(theta2), R[0,0]/np.cos(theta2))

        psi = psi1
        theta = theta1
        phi = phi1

    return psi, theta, phi

print(radians_from_rotation_matrix(image.qvec2rotmat()))


In [ ]:
# https://stackoverflow.com/questions/77655360/direction-vector-correctly-scale-sinradiansx-cosradiansz-with-y-axis
def radians_to_unit_coord(radian_rotaion):
    ''' Includes a correction for different coordinate system like 
        extract_extrinsic_campera_position.fix_rotaion_in_space
    '''
    dir_x = np.sin(radian_rotaion[2]) * np.cos(radian_rotaion[1])
    dir_y = np.sin(radian_rotaion[1])
    dir_z = np.cos(radian_rotaion[2]) * np.cos(radian_rotaion[1])

    vector = (dir_x, dir_y, dir_z)
    uv =  vector / np.linalg.norm(vector)
    rotated_vector = np.zeros_like(uv)
    rotated_vector[0] = -uv[2]
    rotated_vector[1] = uv[0]
    rotated_vector[2] = -uv[1]
    return rotated_vector
    # return uv


print(radians_to_unit_coord(rot1))
print(radians_to_unit_coord(rot2))

In [ ]:
radians = radians_from_rotation_matrix(image.qvec2rotmat())
unit_vector = radians_to_unit_coord(radians)
unit_vector

In [ ]:
unit_vectors = []
for k, image in all_bin_images.items():
    unit_vectors.append(radians_to_unit_coord(radians_from_rotation_matrix(image.qvec2rotmat())))

v = np.array(unit_vectors)
d_v = np.concatenate((np.zeros_like(v), v), axis = 1)

In [ ]:
d_v

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(111, projection='3d')
for vector in d_v:
    v = np.array([vector[3],vector[4],vector[5]])
    vlength=np.linalg.norm(v)
    ax.quiver(vector[0],vector[1],vector[2],vector[3],vector[4],vector[5],
            pivot='tail',length=vlength,arrow_length_ratio=0.3/vlength)

axis_scaler = 1.1
ax.set_xlim([-axis_scaler,axis_scaler])
ax.set_ylim([-axis_scaler,axis_scaler])
ax.set_zlim([-axis_scaler,axis_scaler])
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('z')
plt.show()

# Create intrinsics dict

In [ ]:
all_bin_cameras[image.camera_id]

In [ ]:
'''Camera params are f, cx, cz, k'''
# https://github.com/colmap/colmap/blob/main/src/colmap/sensor/models.h#L281

unit_vectors = []
focal_lengths = []
image_index = []
camera_ids = []
#for idx, cam in all_bin_cameras.items():
for idx, image in all_bin_images.items():
    cam = all_bin_cameras[image.camera_id]
    assert cam.model == 'SIMPLE_RADIAL', f"Camera model is not 'SIMPLE_RADIAL'. Focal length might be at a different parameter."

    unit_vectors.append(radians_to_unit_coord(radians_from_rotation_matrix(image.qvec2rotmat())))
    focal_lengths.append(cam.params[0])
    image_index.append(idx)
    camera_ids.append(image.camera_id)

In [ ]:
# intrinsics.json

camera_intrinsics = {image_index: 
    {
        "unit_vector": vector.tolist(),
        "focal_length": float(length), 
        "camera_id": cam_id
    }
    for image_index, vector, length, cam_id in zip(image_index, unit_vectors, focal_lengths, camera_ids)
}

camera_intrinsics